In [1]:
import torch
import numpy as np

In [2]:
from torch import nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

In [3]:
training_data = datasets.FashionMNIST(
    root = 'FashionMNIST/',
    train = True,
    download = True,
    transform = transforms.ToTensor()
)

In [4]:
test_data = datasets.FashionMNIST(
    root = 'FashionMNIST/',
    train = False,
    download = True,
    transform = transforms.ToTensor()
)

In [5]:
batch_size = 32

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in train_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    # N : Mini-Batch의 크기, C : Chanel의 개수(RGB 이거나 흑백정도), Height, Width
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([32, 1, 28, 28])
Shape of y:  torch.Size([32]) torch.int64


In [143]:
class Convolutional_Encoder(nn.Module):
    def __init__(self):
        super(Convolutional_Encoder, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3,3), stride=1, padding='same')
        self.maxpool = nn.MaxPool2d((2,2))
        self.conv_2 = nn.Conv2d(16, 8, 3, padding='same')
        self.conv_3 = nn.Conv2d(8, 8, 3, padding='same')
        self.zero_pad = nn.ZeroPad2d((7,7,7,7))
        # ----Encoder 부분 여기까지 ----
        
        self.upsample = nn.UpsamplingNearest2d(scale_factor=2)
        self.upsample_7 = nn.UpsamplingNearest2d(7)
        self.conv_4 = nn.Conv2d(8, 16, 3, padding='same')
        self.conv_5 = nn.Conv2d(16, 1, 3, padding='same')
        
        # ----Decoder 부분 여기까지 ----
    def forward(self, x):
        x = self.conv_1(x)
        x = F.relu(x)
        x = self.maxpool(x)
        x = self.conv_2(x)
        x = F.relu(x)
        x = self.maxpool(x)
        x = self.conv_3(x)
        x = F.relu(x)
        x = self.maxpool(x)
        #----encoder-------
        
        x = self.conv_3(x)
        x = F.relu(x)
        x = self.upsample_7(x)
        x = self.conv_3(x)
        x = F.relu(x)
        x = self.upsample(x)
        x = self.conv_4(x)
        x = F.relu(x)
        x = self.upsample(x)
        
        x = self.conv_5(x)
        x = torch.sigmoid(x)
        return x

In [144]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


In [145]:
model = Convolutional_Encoder().to(device)

In [146]:
params = list(model.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([16, 1, 3, 3])


In [150]:
model(s.to(device)).size()

torch.Size([1, 1, 28, 28])

In [151]:
optimizer = torch.optim.RMSprop(model.parameters())
loss_fn = nn.MSELoss()

In [152]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for X, _ in dataloader:# epoch 마다 가중치 갱신
        X = X.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, X)

        # Backpropagation
        # 무조건 optimizer.zero_grad()해줘야 함
        optimizer.zero_grad()
        # 손실함수 미분
        loss.backward()
        # 가중치 갱신
        optimizer.step()

In [180]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, _ in dataloader:
            X = X.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, X).item()
            if np.linalg.norm(np.ravel(X.cpu().numpy(), order='C') - np.ravel(pred.cpu().numpy(), order='C'), axis=-1) < 320:
                correct += 1
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [182]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.206418 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.206418 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.206418 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.206418 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.206418 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.206418 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.206418 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.206418 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.206418 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.206418 

Epoch 11
-------------------------------
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.206418